In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Resizing
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, Concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

# Function to build VGG19 for perceptual loss calculation
def build_vgg19_perceptual_model():
    """
    Load the VGG19 model pre-trained on ImageNet and create a model that outputs the feature maps
    from specific layers used for perceptual loss calculation.
    """
    vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    # Freeze all VGG19 layers
    for layer in vgg19.layers:
        layer.trainable = False

    # Select intermediate layers for perceptual loss (e.g., block4_conv4)
    feature_extractor = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block4_conv4').output)

    return feature_extractor

'''

# Perceptual loss function using VGG19
def perceptual_loss(y_true, y_pred):
    """
    Compute perceptual loss between the ground truth and generated images.
    """
    vgg = build_vgg19_perceptual_model()
    y_true_features = vgg(y_true)
    y_pred_features = vgg(y_pred)
    loss = MeanSquaredError()(y_true_features, y_pred_features)
    return loss

'''

# Build the colorization generator model with transfer learning

def build_colorization_model(input_shape=(256, 256, 1)):
    """
    Builds a colorization model using transfer learning with VGG19 and an encoder-decoder architecture.
    """
    inputs = Input(shape=input_shape)
    
    def grayscale_to_rgb(x):
        return tf.image.grayscale_to_rgb(x)
    
    rgb_inputs = Lambda(grayscale_to_rgb)(inputs)
    
    vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    vgg19_features = Model(inputs=vgg19_base.input, outputs=vgg19_base.get_layer('block2_conv2').output)
    
    vgg_features = vgg19_features(rgb_inputs)
    
    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(vgg_features)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv1)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv2)

    # Decoder
    up1 = UpSampling2D((2, 2))(conv3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
    up2 = UpSampling2D((2, 2))(conv4)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
    up3 = UpSampling2D((2, 2))(conv5)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)

    # Final output layer to produce 3-channel RGB image
    output = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(conv6)
    
    # Ensure output size matches input size
    output = Resizing(256, 256)(output)
    
    model = Model(inputs, output)
    return model




'''

def build_colorization_model(input_shape=(256, 256, 1)):
    """
    Builds a colorization model using transfer learning with VGG19 and an encoder-decoder architecture.
    """
    # Input layer for grayscale images
    inputs = Input(shape=input_shape)

    # Convert grayscale input to 3 channels to fit VGG19 input requirements
    def grayscale_to_rgb(x):
        return tf.image.grayscale_to_rgb(x)
    
    rgb_inputs = Lambda(grayscale_to_rgb)(inputs)

    # VGG19 feature extractor for guiding colorization (use pre-trained weights for transfer learning)
    vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    vgg19_features = Model(inputs=vgg19_base.input, outputs=vgg19_base.get_layer('block2_conv2').output)

    # Extract semantic features from RGB image using VGG19
    vgg_features = vgg19_features(rgb_inputs)

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(vgg_features)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv1)

    # Decoder
    up1 = UpSampling2D((2, 2))(conv2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(up1)
    up2 = UpSampling2D((2, 2))(conv3)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up2)

    # Final output layer to produce 3-channel RGB image
    output = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(conv4)

    # Define the colorization model
    model = Model(inputs=inputs, outputs=output)
    return model


    


'''

'\n\ndef build_colorization_model(input_shape=(256, 256, 1)):\n    """\n    Builds a colorization model using transfer learning with VGG19 and an encoder-decoder architecture.\n    """\n    # Input layer for grayscale images\n    inputs = Input(shape=input_shape)\n\n    # Convert grayscale input to 3 channels to fit VGG19 input requirements\n    def grayscale_to_rgb(x):\n        return tf.image.grayscale_to_rgb(x)\n    \n    rgb_inputs = Lambda(grayscale_to_rgb)(inputs)\n\n    # VGG19 feature extractor for guiding colorization (use pre-trained weights for transfer learning)\n    vgg19_base = VGG19(weights=\'imagenet\', include_top=False, input_shape=(256, 256, 3))\n    vgg19_features = Model(inputs=vgg19_base.input, outputs=vgg19_base.get_layer(\'block2_conv2\').output)\n\n    # Extract semantic features from RGB image using VGG19\n    vgg_features = vgg19_features(rgb_inputs)\n\n    # Encoder\n    conv1 = Conv2D(64, (3, 3), activation=\'relu\', padding=\'same\')(vgg_features)\n    con

: 

In [6]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# Define your image paths
grayscale_images_path = r"C:\Users\dhana\Downloads\SAR Orig\v_2\urban\s1"
colorful_images_path = r"C:\Users\dhana\Downloads\SAR Orig\v_2\urban\s2"

def load_image(image_path, target_size=(256, 256), grayscale=False):
    """
    Load and preprocess an image.
    
    Args:
    - image_path (str): Path to the image.
    - target_size (tuple): Desired size of the image after resizing.
    - grayscale (bool): Whether to load the image in grayscale mode.
    
    Returns:
    - Preprocessed image tensor.
    """
    image = tf.io.read_file(image_path)
    
    if grayscale:
        image = tf.image.decode_image(image, channels=1)  # Load as grayscale
    else:
        image = tf.image.decode_image(image, channels=3)  # Load as RGB
    
    # Resize the image to the target size
    image = tf.image.resize(image, target_size)
    
    # Normalize the image to [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    
    return image

def load_data(batch_size=16):
    """
    Load and preprocess data from the specified directories.
    
    Args:
    - batch_size (int): Number of images per batch.
    
    Returns:
    - sar_images (Tensor): Batch of grayscale SAR images.
    - optical_images (Tensor): Batch of corresponding optical images.
    """
    # Get list of image file paths
    sar_image_paths = sorted([os.path.join(grayscale_images_path, fname) for fname in os.listdir(grayscale_images_path)])
    optical_image_paths = sorted([os.path.join(colorful_images_path, fname) for fname in os.listdir(colorful_images_path)])
    
    # Ensure matching number of SAR and optical images
    assert len(sar_image_paths) == len(optical_image_paths), "Mismatch between number of SAR and optical images."
    
    # Randomly select a batch of images
    indices = tf.random.shuffle(tf.range(len(sar_image_paths)))[:batch_size]
    
    sar_images = [load_image(sar_image_paths[i], target_size=(256, 256), grayscale=True) for i in indices]
    optical_images = [load_image(optical_image_paths[i], target_size=(256, 256), grayscale=False) for i in indices]
    
    # Stack images into a batch
    sar_images = tf.stack(sar_images)
    optical_images = tf.stack(optical_images)

    optical_images = tf.image.resize(optical_images, (256, 256))
    
    return sar_images, optical_images

def perceptual_loss(y_true, y_pred):
    """
    Compute the perceptual loss using VGG19.
    
    Args:
    - y_true (Tensor): Ground truth images.
    - y_pred (Tensor): Generated images by the model.
    
    Returns:
    - loss (Tensor): Perceptual loss computed between VGG features of y_true and y_pred.
    """
    vgg = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    vgg.trainable = False
    
    # Select intermediate layers for feature extraction
    selected_layers = [vgg.get_layer('block3_conv3').output, vgg.get_layer('block4_conv4').output]
    feature_extractor = Model(inputs=vgg.input, outputs=selected_layers)
    
    # Extract features from both ground truth and generated images
    y_true_features = feature_extractor(y_true)
    y_pred_features = feature_extractor(y_pred)
    
    # Compute the perceptual loss as the mean squared error between features
    loss = K.mean(K.square(y_true_features[0] - y_pred_features[0])) + K.mean(K.square(y_true_features[1] - y_pred_features[1]))
    
    return loss

'''generator.compile(optimizer=Adam(0.0008, 0.5), loss=perceptual_loss)
'''


'''generator = build_colorization_model(input_shape=(256, 256, 1))

generator.compile(optimizer=Adam(0.0011, 0.5), loss='mean_squared_error')  # Replace with perceptual_loss if defined

# Training loop
epochs = 10000
batch_size = 16



for epoch in range(epochs):
    sar_images, optical_images = load_data(batch_size)
    
    # Ensure input data size matches model input
    sar_images = tf.image.resize(sar_images, (256, 256))
    optical_images = tf.image.resize(optical_images, (256, 256))
    
    # Train the generator
    loss = generator.train_on_batch(sar_images, optical_images)

    # Print the progress
    print(f"Epoch: {epoch+1}/{epochs}, Loss: {loss}")

    # Save the model periodically
    if ((epoch + 1) > 500) and ((epoch + 1) % 25 == 0):
        generator.save(f"08_09_2024_NEW_colorization_model_epoch_{epoch+1}.h5")

print("Training completed.")'''



# Build and compile the model
generator = build_colorization_model(input_shape=(256, 256, 1))
generator.compile(optimizer=Adam(0.0011, 0.5), loss='mean_squared_error')

# Training loop
epochs = 10000
batch_size = 16

for epoch in range(epochs):
    sar_images, optical_images = load_data(batch_size)
    
    # Ensure input data size matches model input
    sar_images = tf.image.resize(sar_images, (256, 256))
    optical_images = tf.image.resize(optical_images, (256, 256))
    
    # Ensure sar_images have shape (batch_size, 256, 256, 1)
    if sar_images.shape[-1] != 1:
        sar_images = tf.image.rgb_to_grayscale(sar_images)
    
    # Train the generator
    loss = generator.train_on_batch(sar_images, optical_images)

    # Print the progress
    print(f"Epoch: {epoch+1}/{epochs}, Loss: {loss}")

    # Save the model periodically
    if ((epoch + 1) > 500) and ((epoch + 1) % 25 == 0):
        generator.save(f"08_09_2024_NEW_colorization_model_epoch_{epoch+1}.h5")

print("Training completed.")


In [12]:
generator.save(f"08_09_2024_NEW_colorization_model.h5")

In [13]:
"""
V1



import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import os

# Custom function used in the original model
def grayscale_to_rgb(x):
    return tf.image.grayscale_to_rgb(x)

# Load the existing model
custom_objects = {'grayscale_to_rgb': grayscale_to_rgb}
model = load_model("NEW_colorization_model_epoch_1450.h5", custom_objects=custom_objects)

# Data preparation function
def load_and_preprocess_image(grayscale_path, colorful_path, target_size=(256, 256)):
    grayscale_img = tf.io.read_file(grayscale_path)
    grayscale_img = tf.image.decode_png(grayscale_img, channels=1)
    grayscale_img = tf.image.resize(grayscale_img, target_size)
    grayscale_img = grayscale_img / 255.0

    colorful_img = tf.io.read_file(colorful_path)
    colorful_img = tf.image.decode_png(colorful_img, channels=3)
    colorful_img = tf.image.resize(colorful_img, target_size)
    colorful_img = colorful_img / 255.0

    return grayscale_img, colorful_img

# Create a tf.data.Dataset
def create_dataset(grayscale_dir, colorful_dir, batch_size=16):
    grayscale_imgs = sorted([os.path.join(grayscale_dir, fname) for fname in os.listdir(grayscale_dir) if fname.endswith('.png')])
    colorful_imgs = sorted([os.path.join(colorful_dir, fname) for fname in os.listdir(colorful_dir) if fname.endswith('.png')])

    dataset = tf.data.Dataset.from_tensor_slices((grayscale_imgs, colorful_imgs))
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset

# Prepare your data
grayscale_dir = r"C:\Users\dhana\Downloads\SAR Orig\v_2\urban\s1"
colorful_dir = r"C:\Users\dhana\Downloads\SAR Orig\v_2\urban\s2"
batch_size = 16

train_dataset = create_dataset(grayscale_dir, colorful_dir, batch_size)

# Define callbacks
checkpoint = ModelCheckpoint("improved_colorization_model.keras", save_best_only=True, monitor='loss')
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.2, patience=5, min_lr=1e-6)

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(1e-4), loss='mse')

# Fine-tune the model
history = model.fit(
    train_dataset,
    epochs=100,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

# Save the final model
model.save("final_improved_colorization_model.keras")

print("Fine-tuning completed. Model saved as 'final_improved_colorization_model.keras'")



"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 1775-1776: truncated \UXXXXXXXX escape (2181119185.py, line 75)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
from PIL import Image
import os

def grayscale_to_rgb(x):
    return tf.image.grayscale_to_rgb(x)

def preprocess_image(image_path, target_size=(256, 256)):
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = image.resize(target_size)  # Resize to the target size
    image_array = img_to_array(image)  # Convert to numpy array
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    image_array = image_array / 255.0  # Normalize to [0, 1]
    return image_array

def postprocess_image(image_array):
    image_array = image_array.squeeze()  # Remove batch dimension
    image_array = np.clip(image_array, 0, 1)  # Clip values to [0, 1]
    image_array = (image_array * 255).astype(np.uint8)  # Convert to [0, 255] range
    return array_to_img(image_array)  # Convert to PIL Image

def test_colorization_model(model, input_image_path, output_image_path):
    try:
        original_image = Image.open(input_image_path).convert('L')
        preprocessed_image = preprocess_image(input_image_path)
        colorized_image_array = model.predict(preprocessed_image)
        colorized_image = postprocess_image(colorized_image_array[0])
        colorized_image.save(output_image_path)
        
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(original_image, cmap='gray')
        plt.title('Original Grayscale')
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(colorized_image)
        plt.title('Colorized')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
        print(f"Colorized image saved to {output_image_path}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

def test_on_input(model, input_path, output_path):
    if os.path.isfile(input_path):
        test_colorization_model(model, input_path, output_path)
    elif os.path.isdir(input_path):
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        for filename in os.listdir(input_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_file = os.path.join(input_path, filename)
                output_file = os.path.join(output_path, f'colorized_{filename}')
                test_colorization_model(model, input_file, output_file)
    else:
        print(f"Invalid input path: {input_path}")

# Load the model
custom_objects = {'grayscale_to_rgb': grayscale_to_rgb}
generator = tf.keras.models.load_model("BestNEW_colorization_model.h5", custom_objects=custom_objects)

# Usage
input_path = r"C:\Users\dhana\Downloads\SAR Orig\v_2\urban\s1\ROIs1970_fall_s1_8_p36.png"
output_path = r"C:\Users\dhana\AppData\Local\Programs\Python\Python39\op2.png"
test_on_input(generator, input_path, output_path)